<a href="https://colab.research.google.com/github/tusharvatsa32/VisTransformers/blob/main/Code/ViT_pretrained_B_16_imagenet1k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries

In [1]:
!nvidia-smi

Sat May  1 00:34:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    48W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -q einops

In [3]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch import einsum
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
from einops import rearrange, repeat
from einops.layers.torch import Rearrange

In [4]:
print(f"Torch: {torch.__version__}")

Torch: 1.8.1+cu101


In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(11785)

In [6]:
device = 'cuda'

In [7]:
!pip install -q pytorch_pretrained_vit

In [8]:
from pytorch_pretrained_vit import ViT
model = ViT('B_16_imagenet1k', pretrained=True)

Loaded pretrained weights.


In [9]:
model
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 100)

In [10]:
model.to(device)

ViT(
  (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (positional_embedding): PositionalEmbedding1D()
  (transformer): Transformer(
    (blocks): ModuleList(
      (0): Block(
        (attn): MultiHeadedSelfAttention(
          (proj_q): Linear(in_features=768, out_features=768, bias=True)
          (proj_k): Linear(in_features=768, out_features=768, bias=True)
          (proj_v): Linear(in_features=768, out_features=768, bias=True)
          (drop): Dropout(p=0.1, inplace=False)
        )
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (pwff): PositionWiseFeedForward(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
        )
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (drop): Dropout(p=0.1, inplace=False)
      )
      (1): Block(
 

In [11]:
img_size = ((384, 384)) 

transforms_train = transforms.Compose([
    transforms.Resize(img_size),
    transforms.ColorJitter(hue=.05, saturation=.05),
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.RandomRotation(10, interpolation=torchvision.transforms.InterpolationMode.BILINEAR),
    transforms.RandomCrop(img_size, fill=0),
    transforms.RandomAffine(10, scale=(0.8, 1.2)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

transforms_val = transforms.Compose([
    transforms.Resize(img_size),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

## Load Data

In [12]:
train_data = torchvision.datasets.CIFAR100(train=True,download=True,root= "./cifar100/train_data", transform=transforms_train)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=8,
                                          shuffle=True, num_workers=8)

valid_data = torchvision.datasets.CIFAR100(train=False,download=True,root= "./cifar100/test_data", transform=transforms_val)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=4,
                                         shuffle=False, num_workers=8)

Files already downloaded and verified


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Files already downloaded and verified


In [13]:
print(len(train_data), len(train_loader))

50000 6250


In [14]:
print(len(valid_data), len(valid_loader))

10000 2500


### Visual Transformer

### Training

In [15]:
numEpochs = 100
in_features = 3 # RGB channels

learningRate = 0.01
weightDecay = 2e-5

num_classes = len(train_data.classes)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9, nesterov=True)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.4, patience=5, threshold=0.002, verbose=True)

In [16]:
my_acc = []
my_loss = []

In [17]:
# Train!
for epoch in range(numEpochs):
    
    # Train
    model.train()
    train_loss = 0.0
    train_acc = 0.0
    correct = 0

    for batch_num, (x, y) in enumerate(train_loader):
        optimizer.zero_grad()
        
        x, y = x.to(device), y.to(device)

        outputs = model(x)

        correct += (torch.argmax(outputs, axis=1) == y).sum().item()

        loss = criterion(outputs, y.long())
        loss.backward()
        optimizer.step()

        del(outputs)

        train_loss += loss.item()

        if batch_num % 100 == 0:
            print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch, batch_num+1, train_loss/(batch_num+1)))

    train_accuracy = correct / len(train_data)

    # Validate
    model.eval()
    num_correct = 0
    for batch_num1, (x, y) in enumerate(valid_loader):
        x, y = x.to(device), y.to(device)
        outputs = model(x)

        num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()

    val_accuracy = num_correct / len(valid_data)
    my_acc.append(val_accuracy)
    my_loss.append(train_loss/(batch_num+1))
    print('Epoch: {}\t Training Accuracy: {:.4f}\t Validation Accuracy: {:.4f}\t Avg-Loss: {:.4f}'.format(epoch, train_accuracy*100, val_accuracy * 100, train_loss/(batch_num+1)))
    scheduler.step(val_accuracy)

    #torch.save(network.state_dict(),'/content/drive/MyDrive/DL_CMU/HW2_P2/ResNet_Plateau_d3/Net_'+str(epoch)+'_'+str(val_accuracy)+'_checkpoint.t7')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch: 0	Batch: 1	Avg-Loss: 4.7801
Epoch: 0	Batch: 101	Avg-Loss: 4.8242
Epoch: 0	Batch: 201	Avg-Loss: 4.7395
Epoch: 0	Batch: 301	Avg-Loss: 4.6986
Epoch: 0	Batch: 401	Avg-Loss: 4.6760
Epoch: 0	Batch: 501	Avg-Loss: 4.6526
Epoch: 0	Batch: 601	Avg-Loss: 4.6294
Epoch: 0	Batch: 701	Avg-Loss: 4.6027
Epoch: 0	Batch: 801	Avg-Loss: 4.5882
Epoch: 0	Batch: 901	Avg-Loss: 4.5682
Epoch: 0	Batch: 1001	Avg-Loss: 4.5518
Epoch: 0	Batch: 1101	Avg-Loss: 4.5312
Epoch: 0	Batch: 1201	Avg-Loss: 4.5126
Epoch: 0	Batch: 1301	Avg-Loss: 4.4888
Epoch: 0	Batch: 1401	Avg-Loss: 4.4649
Epoch: 0	Batch: 1501	Avg-Loss: 4.4448
Epoch: 0	Batch: 1601	Avg-Loss: 4.4232
Epoch: 0	Batch: 1701	Avg-Loss: 4.4015
Epoch: 0	Batch: 1801	Avg-Loss: 4.3775
Epoch: 0	Batch: 1901	Avg-Loss: 4.3585
Epoch: 0	Batch: 2001	Avg-Loss: 4.3397
Epoch: 0	Batch: 2101	Avg-Loss: 4.3159
Epoch: 0	Batch: 2201	Avg-Loss: 4.2974
Epoch: 0	Batch: 2301	Avg-Loss: 4.2774
Epoch: 0	Batch: 2401	Avg-Loss: 4.2569
Epoch: 0	Batch: 2501	Avg-Loss: 4.2397
Epoch: 0	Batch: 2601	Avg

KeyboardInterrupt: ignored

In [ ]:
# Train!
for epoch in range(numEpochs):
    
    # Train
    model.train()
    train_loss = 0.0
    train_acc = 0.0
    correct = 0

    for batch_num, (x, y) in enumerate(train_loader):
        optimizer.zero_grad()
        
        x, y = x.to(device), y.to(device)

        outputs = model(x)

        correct += (torch.argmax(outputs, axis=1) == y).sum().item()

        loss = criterion(outputs, y.long())
        loss.backward()
        optimizer.step()

        del(outputs)

        train_loss += loss.item()

        if batch_num % 100 == 0:
            print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch, batch_num+1, train_loss/(batch_num+1)))

    train_accuracy = correct / len(train_data)

    # Validate
    model.eval()
    num_correct = 0
    for batch_num1, (x, y) in enumerate(valid_loader):
        x, y = x.to(device), y.to(device)
        outputs = model(x)

        num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()

    val_accuracy = num_correct / len(valid_data)
    my_acc.append(val_accuracy)
    my_loss.append(train_loss/(batch_num+1))
    print('Epoch: {}\t Training Accuracy: {:.4f}\t Validation Accuracy: {:.4f}\t Avg-Loss: {:.4f}'.format(epoch, train_accuracy*100, val_accuracy * 100, train_loss/(batch_num+1)))
    scheduler.step(val_accuracy)

    #torch.save(network.state_dict(),'/content/drive/MyDrive/DL_CMU/HW2_P2/ResNet_Plateau_d3/Net_'+str(epoch)+'_'+str(val_accuracy)+'_checkpoint.t7')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch: 0	Batch: 1	Avg-Loss: 1.9060
Epoch: 0	Batch: 101	Avg-Loss: 0.7388
Epoch: 0	Batch: 201	Avg-Loss: 0.7982
Epoch: 0	Batch: 301	Avg-Loss: 0.8268
Epoch: 0	Batch: 401	Avg-Loss: 0.8307
Epoch: 0	Batch: 501	Avg-Loss: 0.8411
Epoch: 0	Batch: 601	Avg-Loss: 0.8387
Epoch: 0	Batch: 701	Avg-Loss: 0.8453
Epoch: 0	Batch: 801	Avg-Loss: 0.8531
Epoch: 0	Batch: 901	Avg-Loss: 0.8473
Epoch: 0	Batch: 1001	Avg-Loss: 0.8585
Epoch: 0	Batch: 1101	Avg-Loss: 0.8649
Epoch: 0	Batch: 1201	Avg-Loss: 0.8633
Epoch: 0	Batch: 1301	Avg-Loss: 0.8580
Epoch: 0	Batch: 1401	Avg-Loss: 0.8552
Epoch: 0	Batch: 1501	Avg-Loss: 0.8595
Epoch: 0	Batch: 1601	Avg-Loss: 0.8601
Epoch: 0	Batch: 1701	Avg-Loss: 0.8616
Epoch: 0	Batch: 1801	Avg-Loss: 0.8645
Epoch: 0	Batch: 1901	Avg-Loss: 0.8638
Epoch: 0	Batch: 2001	Avg-Loss: 0.8638
Epoch: 0	Batch: 2101	Avg-Loss: 0.8638
Epoch: 0	Batch: 2201	Avg-Loss: 0.8709
Epoch: 0	Batch: 2301	Avg-Loss: 0.8763
Epoch: 0	Batch: 2401	Avg-Loss: 0.8750
Epoch: 0	Batch: 2501	Avg-Loss: 0.8734
Epoch: 0	Batch: 2601	Avg